# This notebook is used to test the Tentacle class without Orchestrator.

### Utility Classes (Robot, PI Controller), etc.


In [ ]:
import RPi.GPIO as GPIO
import gpiozero
import time
import numpy as np
import warnings
from matplotlib import pyplot as plt
from matplotlib.patches import Arrow
from IPython import display
import cv2
import threading
import random

###################################################### Defining our Constants ######################################################
# Encoder pins
# ML = motor left, MR = motor right
GPIO.cleanup()
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

ML_ENC_A = 15 # yellow encoder c 23
ML_ENC_B = 14 # white encoder c 24

MR_ENC_A = 24 # yellow encoder c
MR_ENC_B = 23 # white encoder c

# The number of encoder steps per revolution. This was calculated according to the math that Lacie had in Notion.
MAX_ENC_STEPS = 900

# Motor Pins
ML_IN1 = 17 # IN1 
ML_IN2 = 27 # IN2
ML_ENA = 11 # Used for PWM

MR_IN3 = 22 # IN3
MR_IN4 = 10 # IN4
MR_ENB = 9 # Used for PWM

# Physical dimensions
WHEEL_RADIUS = 0.027 # meters
WHEEL_SEP = 0.229 # meters


###################################################### Class Definitions ######################################################

# Used to implement smoothing dt. Lower alpha means less smoothing, faster response to changes
class ExponentialMovingAverage:
    def __init__(self, alpha):
        self.alpha = alpha
        self.value = None

    def update(self, new_value):
        if self.value is None:
            self.value = new_value
        else:
            self.value = (1 - self.alpha) * new_value + self.alpha * self.value
        return self.value
    

# Used to implement velocity smoothing. Lower tau means less smoothing, faster response to changes.
class LowPassFilter:
    def __init__(self, tau):
        self.tau = tau
        self.previous_value = None
        self._last_smoothed_value = None

    def update(self, new_value, dt):
        if self.previous_value is None:
            self.previous_value = new_value
            self._last_smoothed_value = new_value
        else:
            alpha = dt / (self.tau + dt)
            self._last_smoothed_value = alpha * new_value + (1 - alpha) * self.previous_value
            self.previous_value = self._last_smoothed_value
        return self._last_smoothed_value

    @property
    def value(self):
        if self._last_smoothed_value is None:
            return 0
        return self._last_smoothed_value
    

class DiffDriveRobot:
    def __init__(
            self, 
            dt=0.05, 
            real_time=False,
            tau = 0.1,
            wheel_radius=WHEEL_RADIUS, 
            wheel_sep=WHEEL_SEP,
            max_enc_steps=MAX_ENC_STEPS,
            ml_pwm=ML_ENA,
            mr_pwm=MR_ENB,
            ml_in1=ML_IN1,
            ml_in2=ML_IN2,
            mr_in3=MR_IN3,
            mr_in4=MR_IN4,
            ml_encA=ML_ENC_A,
            ml_encB=ML_ENC_B,
            mr_encA=MR_ENC_A,
            mr_encB=MR_ENC_B
    ):
        self.x = 0.0  # x-position, meters
        self.y = 0.0  # y-position, meters
        self.th = 0.0  # orientation, angle in radians
        
        self.wl = 0.0  # rotational velocity left wheel, rad/s
        self.wr = 0.0  # rotational velocity right wheel, rad/s
        
#         self.wl_prev = 0
#         self.wr_prev = 0
        
        self.wl_smoothed = LowPassFilter(tau)
        self.wr_smoothed = LowPassFilter(tau)
                
        self.dt = dt # time delta in seconds. The control loop runs every dt. Faster means the control loop runs more often. We can increase this to reduce CPU load on our robot.
        self.r = wheel_radius # wheel radius in meters.
        self.l = wheel_sep # wheel separation in meters
        self.max_enc_steps = max_enc_steps # steps in the encoder per revolution
        
        self.real_time = real_time # if True, the wheel velocity measurements consider the actual time elapsed between measurements. If False, it uses the predefined dt value.
        self.last_update = None # last time the control loop ran

        # Pin numbers
        self.motor_L_in1 = ml_in1 # Input 1 (motor left)
        self.motor_L_in2 = ml_in2 # Input 2 (motor left)
        self.motor_R_in3 = mr_in3 # Input 3 (motor right)
        self.motor_R_in4 = mr_in4 # Input 4 (motor right)
        GPIO.setup(self.motor_L_in1, GPIO.OUT)
        GPIO.setup(self.motor_L_in2, GPIO.OUT)
        GPIO.setup(self.motor_R_in3, GPIO.OUT)
        GPIO.setup(self.motor_R_in4, GPIO.OUT)      
        
        # Initialize encoders
        self.ML_ENC = gpiozero.RotaryEncoder(a=ml_encA, b=ml_encB, max_steps=max_enc_steps, wrap=True)
        self.MR_ENC = gpiozero.RotaryEncoder(a=mr_encA, b=mr_encB, max_steps=max_enc_steps, wrap=True)
        self.ml_enc_steps = 0 # left motor encoder value (AKA shaft angle) in steps. Note, this is NOT in radians. It counts from 0 -> MAX_ENC_STEPS -> -MAX_ENC_STEPS (wraps around to -ve value)
        self.mr_enc_steps = 0 # right motor encoder value (AKA shaft angle) in steps. 

        # Initialize motor control pins
        GPIO.setup(ml_pwm, GPIO.OUT)
        GPIO.setup(mr_pwm, GPIO.OUT)
        self.motor_L_pwm = GPIO.PWM(ml_pwm, 1000)  # 1000 Hz frequency. This works well for our motor. Remember that motor speed is controlled by varying the duty cycle of our PWM frequency, and not the frequency itself. 
        self.motor_R_pwm = GPIO.PWM(mr_pwm, 1000)
        self.motor_L_pwm.start(0)
        self.motor_R_pwm.start(0)
    '''
    This method calculates the change in encoder steps between the current and previous time step.
    The logic might seem complicated but this is to deal with the wraparound in value (from 0 -> MAX_ENC_STEPS -> -MAX_ENC_STEPS) that the encoder values have.
    e.g. for the first rotation, the encoder goes from 0 to 3600, then for every rotation thereafter, it goes from -3600 to 3600, then wraps around again.
    '''
    @property
    def pose(self):
        return (self.x, self.y, self.th)
    
    def get_encoder_delta(self, curr_value, prev_value):
        raw_delta = curr_value - prev_value
        delta = raw_delta
        wraparound_thresh = np.ceil((2*self.max_enc_steps + 1) / 2)
        
        if raw_delta > wraparound_thresh:
            delta = raw_delta - (2*self.max_enc_steps+1)
        elif raw_delta < -wraparound_thresh:
            delta = raw_delta + (2*self.max_enc_steps+1)
        
        return delta
            
    def get_dt(self):
        #TODO: Implement smoothing for dt - potentially moving average
        if not self.real_time:
            return self.dt
        
        now = time.time()
        # print(f"Last update: {self.last_update}")
        if self.last_update is None:
            self.last_update = now
            return self.dt
        
        dt = now - self.last_update
        self.last_update = now # this needs to be in between the above and below lines. Don't move it 
        return dt
    
    # Wheel velocities in radians/sec
    '''
    This method reads the encoder values and calculates the wheel velocities in rad/s.
    It uses the get_encoder_delta method to calculate the change in encoder steps between the current and previous time step.
    It then converts this change in steps to radians, and then to radians per second.
    '''
    def read_wheel_velocities(self, dt):
        
        ml_enc_now, mr_enc_now = self.ML_ENC.steps, self.MR_ENC.steps

        # Calculate change in steps, accounting for wrap-around
        ml_enc_delta = self.get_encoder_delta(ml_enc_now, self.ml_enc_steps)
        mr_enc_delta = self.get_encoder_delta(mr_enc_now, self.mr_enc_steps)

        # Convert step change to radians
        ml_delta_rad = ml_enc_delta / self.max_enc_steps * 2 * np.pi 
        mr_delta_rad = mr_enc_delta / self.max_enc_steps * 2 * np.pi

        # Calculate velocities
#         self.wl_prev = self.wl
#         self.wr_prev = self.wr
        
        self.wl = ml_delta_rad / dt # rad/s
        self.wr = mr_delta_rad / dt # rad/s
        
        # Calculate smoothed velocities
#         self.wl_smoothed = self.tau/(self.tau+dt) * self.wl_prev + dt/(self.tau+dt) * self.wl
#         self.wr_smoothed = self.tau/(self.tau+dt) * self.wr_prev + dt/(self.tau+dt) * self.wr
        
        self.wl_smoothed.update(self.wl, dt)
        self.wr_smoothed.update(self.wr, dt)
        
        # Update previous steps
        self.ml_enc_steps = ml_enc_now
        self.mr_enc_steps = mr_enc_now

        return self.wl, self.wr
  
    '''
    This method sets the motor speed based on the duty cycle provided. 
    It also sets the direction of the motor based on the sign of the duty cycle.
    The duty cycle is the percentage of time the motor is on, and it MUST be a value between -100 and 100.
    '''
    def set_motor_speed(self, left_duty_cycle, right_duty_cycle):      
        # Set direction
        GPIO.output(self.motor_L_in1, GPIO.HIGH if left_duty_cycle >= 0 else GPIO.LOW)
        GPIO.output(self.motor_L_in2, GPIO.LOW if left_duty_cycle >= 0 else GPIO.HIGH)
        GPIO.output(self.motor_R_in3, GPIO.HIGH if right_duty_cycle >= 0 else GPIO.LOW)
        GPIO.output(self.motor_R_in4, GPIO.LOW if right_duty_cycle >= 0 else GPIO.HIGH)
            
        # Set speed
        self.motor_L_pwm.ChangeDutyCycle(abs(left_duty_cycle))
        self.motor_R_pwm.ChangeDutyCycle(abs(right_duty_cycle))


    '''
    This method calculates the linear and angular velocity of the robot based on the wheel velocities.
    It uses the formulae for differential drive robots to calculate the linear and angular velocity.
    This is pretty much identical to Michael's code in the ECE4191 repo.
    '''
    def base_velocity(self, wl, wr):
        v = (wl * self.r + wr * self.r) / 2.0 # linear velocity, m/s, +ve is forward
        w = (wl * self.r - wr * self.r) / self.l # angular velocity, rad/s, +ve is CCW. Note that the negative sign
        # is due to the way the motors are oriented, and so we need it to 'correct' our w calculation
        # so that +ve w is CCW, adhering to convention.
        return v, w
    
    
    """
    This method updates the robot's pose (x, y, theta) based on the wheel velocities.
    It uses the base_velocity method to calculate the linear and angular velocity of the robot.
    It then uses these velocities to update the robot's pose (x, y, theta) based on the kinematic equations.
    """
    def pose_update(self, duty_cycle_ml, duty_cycle_mr):
        # TODO: remove the call to get_dt here, relocate it to the read_wheel_velocities method, and remove the need for it in the pose_update method (below) by having read_wheel_velocities return dt, or the distance travelled (dt*v) directly.
        dt = self.get_dt()
        
        self.set_motor_speed(duty_cycle_ml, duty_cycle_mr)

        wl, wr = self.read_wheel_velocities(dt) # get wheel velocities in rad/s
        v, w = self.base_velocity(wl, wr)
        
        self.x = self.x + dt * v * np.cos(self.th)
        self.y = self.y + dt * v * np.sin(self.th)
        self.th = self.th + w * dt
        
        return self.x, self.y, self.th
    
    
class RobotController:
    def __init__(self, Kp=8, Ki=9, dt=0.05, wheel_radius = 0.028, wheel_sep = 0.222, integral_windup=True, real_time=False):
        self.Kp = Kp            # Proportional gain
        self.Ki = Ki            # Integral gain
        self.dt = dt            # Time step
        
        self.r = wheel_radius
        self.l = wheel_sep
        
        self.min_output = -100  # Minimum duty cycle (-100%)
        self.max_output = 100  # Maximum duty cycle (100%)
        
        self.ml_integral = 0       # Integral term, motor left
        self.mr_integral = 0       # Integral term, motor right
        self.ml_last_update, self.mr_last_update = None, None  # Last update time

        self.anti_integral_windup = integral_windup  # Anti-windup flag
        self.real_time = real_time

    """
    Gets the time of the last measurement/update for the motor ml or mr
    """
    def get_dt(self, motor=None):
        #TODO: Implement smoothing for dt - potentially moving average
        if motor not in ['ml', 'mr']:
            raise ValueError("Motor must be 'ml' or 'mr'.")

        if not self.real_time:
            return self.dt
        
        last_update = f"{motor}_last_update"
        now = time.time()
#         print(f"Last update: {getattr(self, last_update)}")
        if getattr(self, last_update) is None:
            setattr(self, last_update, now)
            return self.dt
        
        dt = now - getattr(self, last_update)
        setattr(self, last_update, now)
        return dt


    def compute(self, w_target, w_actual, motor=None):
        # motor should be either "ml" or "mr" for left or right motor
        if motor not in ['ml', 'mr']:
            raise ValueError("Motor must be 'ml' or 'mr'.")
        integral_attr = f"{motor}_integral"
        last_update = f"{motor}_last_update"
        
        dt = self.get_dt(motor)
        
        error = w_target - w_actual # Calculate the error
        P_out = self.Kp * error # Proportional term
        I_out = self.Ki * getattr(self,integral_attr) # Integral term
        raw_output = P_out + I_out

        if self.anti_integral_windup:
            # Anti-windup - only integrate if output is not saturated
            if self.min_output < raw_output < self.max_output:
                setattr(self, integral_attr, getattr(self, integral_attr) + error * dt)
                # equiv. to self.ml_integral += error * self.dt or self.mr_integral += error * self.dt
        else:
            setattr(self, integral_attr, getattr(self, integral_attr) + error * dt)
                        
            
        return np.clip(raw_output, self.min_output, self.max_output) # Clamp the output to the min/max duty cycle limits
        
    
    def diff_drive_control(self, v_desired, w_desired, wl_actual, wr_actual):
        # v_desired: m/s
        # w_desired (rotation), wl_actual, w_actual (rotation): rad/s
        
        if v_desired == 0 and w_desired == 0:
            return 0, 0, 0, 0
        
        # Calculate desired wheel angular velocities
        wl_desired = (v_desired + self.l*w_desired/2)/self.r
        wr_desired = (v_desired - self.l*w_desired/2)/self.r
        
#         print(f"wl_des (rad/s): {wl_desired:.2f}, wr_des: {wr_desired:.2f}\nwl_des (rps): {wl_desired/(2*np.pi):.2f}, wr_des: {wr_desired/(2*np.pi):.2f}")

        # Compute duty cycles for left and right wheels
        duty_cycle_l = self.compute(wl_desired, wl_actual, 'ml')
        duty_cycle_r = self.compute(wr_desired, wr_actual, 'mr')
        return duty_cycle_l, duty_cycle_r, wl_desired, wr_desired

    

### PASTE IN YOUR TENTACLE PLANNER HERE

In [ ]:
# PASTE YOUR TENTACLE PLANNER HERE

Run the below cell ONCE to initialise the robot.
- After you run this cell once, you can rerun the below control loop as many times as you want without needing to restart kernel


In [ ]:

GPIO.setwarnings(False)
robot = DiffDriveRobot(0.05, real_time=True)

### Control Loop - You can rerun this as many times as you'd like without needing to restart the kernel. 

In [ ]:
# Here are some example position commands we can put in.

# goal = (0.5, 0.5, 0)
# goal = (0.5, 0.5, np.pi/2)
# goal = (0, 1, 0)
goal = (1, 0, 0)




# Initialising controller, logging variables 
# Controller Parameters
controller = RobotController(real_time=True) # using default Kp = 8, Ki=9
planner = TentaclePlanner(max_linear_velocity=0.1, max_angular_velocity=0.5) # remember to paste in the class in the cell where I told you


robot.x = 0
robot.y = 0
robot.th = 0
robot.wl = 0
robot.wr = 0

poses = []
goal_positions = []
velocities = []
desired_velocities = []
duty_cycle_commands = []
error_sums = []
errors = []
actual_dts = []



try:
    goal_positions.append([0, 0, 0])
    goal_positions.append([goal[0],goal[1],goal[2]])
    start_time = time.time()
    while True: 
        inputs = planner.get_control_inputs(*goal, *robot.pose, strategy="tentacles")
        duty_cycle_l, duty_cycle_r, wl_desired, wr_desired = controller.diff_drive_control(
                    inputs['linear_velocity'], 
                    inputs['angular_velocity'], 
                    robot.wl_smoothed.value, 
                    robot.wr_smoothed.value
                )
        x, y, th = robot.pose_update(duty_cycle_l, duty_cycle_r)
        display.clear_output(wait=True)


        
        # Log data
        # Here, I record and log everything so we can graph it.
        poses.append([x,y,th])
        duty_cycle_commands.append([duty_cycle_l,duty_cycle_r])
        velocities.append([robot.wl,robot.wr])
        desired_velocities.append([wl_desired, wr_desired])
        time.sleep(robot.dt)

except KeyboardInterrupt:
    print("Stopping robot")
    robot.set_motor_speed(0, 0)

    duration = time.time() - start_time

    # Plot robot data
#     display.clear_output(wait=True)
    fig = plt.figure(figsize=(15,9))
    
    plt.subplot(2, 2, 1)
    plt.plot(np.array(poses)[:,0],np.array(poses)[:,1], 'b', label='Robot Path')
    plt.plot(np.array(goal_positions)[:,0],np.array(goal_positions)[:,1],'ro', label='Goal Position')
    plt.plot(x,y,'k',marker='+')
    plt.quiver(x,y,0.1*np.cos(th),0.1*np.sin(th))
    plt.xlabel('x-position (m)')
    plt.ylabel('y-position (m)')
    plt.title(f"Ran for {duration:.2f} secs")
#     plt.axis('equal')
    plt.grid()

    plt.subplot(2, 2, 2)
    duty_cycle_commands = np.array(duty_cycle_commands)
    plt.plot(duty_cycle_commands[:, 0], label='Left Wheel')
    plt.plot(duty_cycle_commands[:, 1], label='Right Wheel')
    plt.xlabel('Time Step')
    plt.ylabel('Duty Cycle')
    plt.title('Duty Cycle Commands Over Time')
    plt.legend() 
    plt.grid()
    
    plt.subplot(2, 2, 3)
    velocities = np.array(velocities)
    desired_velocities = np.array(desired_velocities)
    plt.plot(velocities[:, 0], label='Left Wheel')
    plt.plot(velocities[:, 1], label='Right Wheel')
    plt.plot(desired_velocities[:, 0], label='Desired Left Wheel')
    plt.plot(desired_velocities[:, 1], label='Desired Right Wheel')
    plt.xlabel('Time Step')
    plt.ylabel('Wheel Velocity (rad/s)')
    plt.title('Wheel Velocity vs. Time')
    plt.legend() 
    plt.grid()
   
        
    fig.tight_layout()

    display.display(plt.gcf())

#     GPIO.cleanup()

    """
    Hopefully the graphs are self explanatory. If not, please ask me. I'm happy to explain anything.
    """
